#### In this Jupyter Notebook we will display the results after training of the model

##### Importing the libraries

In [ ]:
import os
from os.path import exists
from glob import glob
import torch
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss
from monai.data import Dataset, CacheDataset,DataLoader
from monai.utils import set_determinism
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ScaleIntensityRanged,
    RandAffined,
    RandRotated,
    RandGaussianNoised,
    CropForegroundd,
    Orientationd,
    Resized,
    ToTensord,
    Spacingd,
)

##### Setting the path to the working directories

In [ ]:
# The input paths for the prepared nifti files
nif_path = ['data_set_group_nif/nif_files_testing/images',
            'data_set_group_nif/nif_files_testing/labels',
            'data_set_group_nif/nif_files_training/images',
            'data_set_group_nif/nif_files_training/labels',]

##### Define the function for data preprocessing

In [ ]:
def preprocess_data(data_path, batch_size=1, spatial_size=(256, 256, 16)):

    set_determinism(seed=0)

    # Create the dataset
    test_data = sorted(glob(data_path[0] + f'/*'))
    test_labels = sorted(glob(data_path[1] + f'/*'))

    train_data = sorted(glob(data_path[2] + f'/*'))
    train_labels = sorted(glob(data_path[3] + f'/*'))

    train_files = [{"image": image_name, "label": label_name} for image_name, label_name in zip(train_data, train_labels)]
    test_files = [{"image": image_name, "label": label_name} for image_name, label_name in zip(test_data, test_labels)]

    # Transforms for the training with data augmentation
    train_transforms = Compose(# Compose transforms together
        [
            LoadImaged(keys=["image", "label"]), # Load the images
            EnsureChannelFirstd(keys=["image", "label"]), # Ensure the channel is the first dimension of the image
            Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")), # Resample the images
            Orientationd(keys=["image", "label"], axcodes="RAS"), # Change the orientation of the image
            ScaleIntensityRanged(keys=["image"], a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),# Change the contrast of the image and gives the image pixels,
                                                                                                            #values between 0 and 1
            CropForegroundd(keys=["image", "label"], source_key="image"), # Crop foreground of the image
            RandAffined(keys=['image', 'label'], prob=0.5, translate_range=10), # Randomly shift the image
            RandRotated(keys=['image', 'label'], prob=0.5, range_x=10.0), # Randomly rotate the image
            RandGaussianNoised(keys='image', prob=0.5), # Add random noise to the image
            Resized(keys=["image", "label"], spatial_size=spatial_size), # Resize the image
            ToTensord(keys=["image", "label"]), # Convert the images to tensors
        ]
    )

    # Transforms for the testing
    test_transforms = Compose(# Compose transforms together
        [
            LoadImaged(keys=["image", "label"]), # Load the images
            EnsureChannelFirstd(keys=["image", "label"]), # Ensure the channel is the first dimension of the image
            Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")), # Resample the images
            Orientationd(keys=["image", "label"], axcodes="RAS"), # Change the orientation of the image
            ScaleIntensityRanged(keys=["image"], a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),# Change the contrast of the image and gives the image pixels,
                                                                                                            #values between 0 and 1
            CropForegroundd(keys=["image", "label"], source_key="image"), # Crop foreground of the image
            Resized(keys=["image", "label"], spatial_size=spatial_size), # Resize the image
            ToTensord(keys=["image", "label"]), # Convert the images to tensors
        ]
    )

    # Create the datasets
    train_ds = CacheDataset(data=train_files, transform=train_transforms)
    train_loader = DataLoader(train_ds, batch_size=batch_size)

    test_ds = CacheDataset(data=test_files, transform=test_transforms)
    test_loader = DataLoader(test_ds, batch_size=batch_size)

    '''for data in train_loader:
        print(data["image"].shape)
        print(data["label"].shape)

    for data in test_loader:
        print(data["image"].shape)
        print(data["label"].shape)'''

    return train_loader, test_loader

##### Preprocess the data

In [ ]:
data_in = preprocess_data(nif_path, batch_size=1, spatial_size=(512, 512, 16))

##### Setting the device for training

In [ ]:
# We do the training on the GPU
device = torch.device("cuda:0")
print(device)

##### Initialize the model

In [ ]:
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

##### Initialize the loss function and the optimizer

In [ ]:
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

##### Define the training loop

In [ ]:
def train(
        model,
        data_in,
        loss_function,
        optimizer,
        max_epochs,
        model_dir,
        test_interval=1,
        device=torch.device('cuda:0')
    ):
    best_metric = -1
    best_metric_epoch = -1
    save_loss_train = []
    save_loss_test = []
    save_metric_train = []
    save_metric_test = []
    train_loader, test_loader = data_in

    for epoch in range(max_epochs):
        model.train()
        train_epoch_loss = 0
        train_step = 0
        train_epoch_metric = 0

        for batch_data in train_loader:
            train_step += 1
            volumes = batch_data["image"]
            labels = batch_data["label"]
            labels = labels != 0
            volumes = volumes.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(volumes)

            train_loss = loss_function(outputs, labels)

            train_loss.backward()
            optimizer.step()

            train_epoch_loss += train_loss.item()
            train_metric = dice_metric(outputs, labels)
            train_epoch_metric += train_metric

            print(f"{epoch + 1}/{max_epochs} and {train_step}/{len(train_loader) // train_loader.batch_size} => train_loss: {train_loss.item():.4f} and train_metric: {train_metric:.4f}")

        print('Saving training data after epoch: ' + str(epoch + 1))
        train_epoch_loss /= train_step
        print(f"epoch {epoch + 1} average training loss: {train_epoch_loss:.4f}")
        save_loss_train.append(train_epoch_loss)
        np.save(os.path.join(model_dir, 'train_loss.npy'), save_loss_train)

        train_epoch_metric /= train_step
        print(f"epoch {epoch + 1} average training metric: {train_epoch_metric:.4f}")
        save_metric_train.append(train_epoch_metric)
        np.save(os.path.join(model_dir, 'train_metric.npy'), save_metric_train)

        if (epoch+1) % test_interval == 0:
            model.eval()
            with torch.no_grad():
                test_epoch_loss = 0
                test_metric = 0
                test_step = 0
                test_epoch_metric = 0

                for test_data in test_loader:
                    test_step += 1
                    volumes = test_data["image"]
                    labels = test_data["label"]
                    labels = labels != 0
                    volumes = volumes.to(device)
                    labels = labels.to(device)

                    outputs = model(volumes)

                    test_loss = loss_function(outputs, labels)

                    test_epoch_loss += test_loss.item()
                    test_metric = dice_metric(outputs, labels)
                    test_epoch_metric += test_metric

                    print(f"{epoch + 1}/{max_epochs} and {test_step}/{len(test_loader) // test_loader.batch_size} => test_loss: {test_loss.item():.4f} and test_metric: {test_metric:.4f}")

                print('Saving testing data after epoch: ' + str(epoch + 1))
                test_epoch_loss /= test_step
                print(f"epoch {epoch + 1} average testing loss: {test_epoch_loss:.4f}")
                save_loss_test.append(test_epoch_loss)
                np.save(os.path.join(model_dir, 'test_loss.npy'), save_loss_test)

                test_epoch_metric /= test_step
                print(f"epoch {epoch + 1} average testing metric: {test_epoch_metric:.4f}")
                save_metric_test.append(test_epoch_metric)
                np.save(os.path.join(model_dir, 'test_metric.npy'), save_metric_test)

                if test_epoch_metric > best_metric:
                    best_metric = test_epoch_metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), os.path.join(model_dir, "best_metric_model.pth"))

                print(f"current epoch: {epoch + 1} current test Dice coefficient: {test_epoch_metric:.4f}"
                    f"\nbest metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

    print(f"train completed => best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

##### Train the model

In [ ]:
os.makedirs('post_training', exist_ok=true)

train(model,
      data_in,
      loss_function,
      optimizer,
      10,
      '../post_training_best',
      test_interval=100,
      device=device
)
